In [1]:
import pygame
import pygame_gui

# scene boilerplate
class SceneBase:
    def __init__(self):
        self.next = self
        self.dt = 0
        self.player_pos = pygame.Vector2(640, 360) #hardcoded for now, i dont wanna pass screen everywhere atm
    
    def ProcessInput(self, events, keys, screen, dt, uiManager, uiElements):
        print("uh-oh, you didn't override this in the child class")

    def Update(self, dt, screen, uiManager):
        print("uh-oh, you didn't override this in the child class")

    def Render(self, screen, uiManager):
        print("uh-oh, you didn't override this in the child class")

    def SwitchToScene(self, next_scene):
        self.next = next_scene

def runGame(width, height, fps, start_scene):
    pygame.init()
    screen = pygame.display.set_mode((width, height))
    pygame.display.set_caption("Pygame GUI Demo")
    clock = pygame.time.Clock()
    uiManager = pygame_gui.UIManager((width, height))
    dt = 0
    # ui elements, centered on the screen
    # later we should probably find a way to encapsulate this, so ui can be easily passed about
    uiWidth = 100
    uiHeight = 50
    uiStart = (width - uiWidth) // 2
    uiEnd = (height - uiHeight) // 2
    startButton = pygame_gui.elements.UIButton(relative_rect=pygame.Rect((uiStart, uiEnd), (uiWidth, uiHeight)), text='Start', manager=uiManager)
    exitButton = pygame_gui.elements.UIButton(relative_rect=pygame.Rect((uiStart, uiEnd + 100), (uiWidth, uiHeight)), text='Exit', manager=uiManager)
    uiElements = [startButton, exitButton]
    active_scene = start_scene

    while active_scene != None: #end state is None
        dt = clock.tick(fps) / 1000 #delta time
        # basic event filtering for exits
        keys = pygame.key.get_pressed()
        filtered_events = []
        for event in pygame.event.get():
            quit = False
            if event.type == pygame.QUIT:
                quit = True
            elif event.type == pygame.KEYDOWN:
                alt_pressed = keys[pygame.K_LALT] or keys[pygame.K_RALT]
                if keys[pygame.K_ESCAPE]:
                    quit = True
                elif keys[pygame.K_F4] and alt_pressed:
                    quit = True
            if quit:
                active_scene.Terminate()
            else:
                filtered_events.append(event)
        
        # handle scene (input, update, render)
        active_scene.ProcessInput(filtered_events, keys, screen, dt, uiManager, uiElements)
        active_scene.Update(dt, screen, uiManager)
        active_scene.Render(screen, uiManager)

        active_scene = active_scene.next #switch to next scene, if necessary
        pygame.display.flip()

    # when the game loop ends, quit pygame and call sys.exit
    pygame.quit()

# title screen
class TitleScene(SceneBase):
    def __init__(self):
        SceneBase.__init__(self)
    
    def ProcessInput(self, events, keys, screen, dt, uiManager, uiElements):
        for event in events:
            if event.type == pygame_gui.UI_BUTTON_PRESSED:
                if event.ui_element == uiElements[0]:
                    print("Start button pressed")
                    self.SwitchToScene(GameScene())
                elif event.ui_element == uiElements[1]:
                    print("Exit button pressed")
                    self.SwitchToScene(None)
            uiManager.process_events(event)
    
    def Update(self, dt, screen, uiManager):
        uiManager.update(dt)
    
    def Render(self, screen, uiManager):
        screen.fill((255,255,255))
        # draw title text
        font = pygame.font.Font(None, 74)
        text = font.render("Welcome to Pygame!", True, (0, 0, 0))
        text_rect = text.get_rect()
        text_x = screen.get_width() / 2 - text_rect.width / 2 
        text_y = screen.get_height() / 2 - text_rect.height / 2 - 100
        screen.blit(text, [text_x, text_y])
        uiManager.draw_ui(screen)

    def Terminate(self):
        self.SwitchToScene(None)

# game screen
#gonna port the first test into a scene, so we have a little interactive demo
class GameScene(SceneBase):
    def init(self):
        SceneBase.init(self) # call the parent class's init (just set next to self!)
        # create variables for player position and dt
        self.player_pos = pygame.Vector2(640, 360) #hardcoded for now, i dont wanna pass screen everywhere atm
    
    def ProcessInput(self, events, pressed_keys, screen, dt, uiManager, uiElements):
        # don't worry about filtering, that's in the main loop
        # need to pass in screen to get bounds of the window--we'll find a better way to do this with time
        # here, we just want to check if the arrow/wasd keys are pressed
        # if they are, we'll move the player: all ifs so diagonal movement works
        if pressed_keys[pygame.K_w] or pressed_keys[pygame.K_UP]:
            #print("up" )
            self.player_pos.y -= 300 * dt
        if pressed_keys[pygame.K_s] or pressed_keys[pygame.K_DOWN]:
            #print("down")
            self.player_pos.y += 300 * dt
        if pressed_keys[pygame.K_a] or pressed_keys[pygame.K_LEFT]:
            #print("left")
            self.player_pos.x -= 300 * dt
        if pressed_keys[pygame.K_d] or pressed_keys[pygame.K_RIGHT]:
            #print("right")
            self.player_pos.x += 300 * dt
        
        # finally, lets bound the player to the screen--keep in mind the circle has a radius of 40
        self.player_pos.x = max(40, min(self.player_pos.x, screen.get_width() - 40))
        self.player_pos.y = max(40, min(self.player_pos.y, screen.get_height() - 40))

    def Update(self, dt, screen, uiManager):
        pass

    def Render(self, screen, uiManager):
        screen.fill((0, 0, 255)) # fill the screen with blue
        pygame.draw.circle(screen, (128, 0, 128), self.player_pos, 40) # draw a purple circle at the player's position
        

    def Terminate(self):
        self.SwitchToScene(None) # switch to None to end the game



runGame(1280, 720, 60, TitleScene()) #start the game with the title screen

pygame-ce 2.3.2 (SDL 2.26.5, Python 3.7.1)
Start button pressed
